In [ ]:
from collections import defaultdict

In [ ]:
raw_read_1 = "/media/eternus1/nfs/projects/nkureev/mini_reads/amp_res_1.fastq" # 417842 ридов в сумме
raw_read_2 = "/media/eternus1/nfs/projects/nkureev/mini_reads/amp_res_2.fastq"

read_1_trimmomatic = "/media/eternus1/nfs/projects/nkureev/mini_reads/trimmomatic/trimmomatic_done_pairedPE_1.fq"
read_2_trimmomatic = "/media/eternus1/nfs/projects/nkureev/mini_reads/trimmomatic/trimmomatic_done_pairedPE_2.fq"

reads_1_v2trim = "/media/eternus1/nfs/projects/nkureev/mini_reads/v2trim/minireads.trim_1.fastq"
reads_2_v2trim = "/media/eternus1/nfs/projects/nkureev/mini_reads/v2trim/minireads.trim_2.fastq"

reads_1_fastp = "/media/eternus1/nfs/projects/nkureev/mini_reads/fastp/fastp_done_1.fq"
reads_2_fastp = "/media/eternus1/nfs/projects/nkureev/mini_reads/fastp/fastp_done_2.fq"

reads_1_trimgalore = "/media/eternus1/nfs/projects/nkureev/mini_reads/trim_galore/trim_galore_done/amp_res_1_val_1.fq"
reads_2_trimgalore = "/media/eternus1/nfs/projects/nkureev/mini_reads/trim_galore/trim_galore_done/amp_res_2_val_2.fq"

reads_1_prinseq = "/media/eternus1/nfs/projects/nkureev/mini_reads/prinseq/prinseq_done.fq_good_out_R1.fastq"
reads_2_prinseq = "/media/eternus1/nfs/projects/nkureev/mini_reads/prinseq/prinseq_done.fq_good_out_R2.fastq"

reads_1_adapterremoval = "/media/eternus1/nfs/projects/nkureev/mini_reads/AdapterRemoval/AR_done_1.fq"
reads_2_adapterremoval ="/media/eternus1/nfs/projects/nkureev/mini_reads/AdapterRemoval/AR_done_2.fq"

In [ ]:
class FastqObj(object):

    def __init__(self, head, seq, strain, qual, phred33=False, forvard=True):
        self.head = head.strip()
        self.seq = seq.strip()
        self.qual = qual.strip()
        self.strain = strain.strip()
        self.head_n = head_n.strip()
        if forvard:
            self.id = head.strip().replace("#", " ").split()[0].replace("@","") + "_1"
        else:
            self.id = head.strip().replace("#", " ").split()[0].replace("@","") + "_2"

In [ ]:
def fastq_reader(fastq_file):
    with open(fastq_file) as fh:
        while True:
            try:
                head = fh.readline()
                if head1 == '':
                    break
                seq = fh.readline()
                strain = fh.readline()
                qual = fh.readline()
                fastq_obj = FastqObj(head, seq, strain, qual)
                yield fastq_obj
            except:
                break

In [ ]:
# returns table with 0 if read is the same and 1 if read was trimmed
# columns is tools which was compared to base_tool

def reads_comparer(base_tool_obj, compared_tools_list):
    number_of_tools = len(compared_tools_list)
    while True:
        try:
            reads_to_compare = defaultdict(list)
            base_next = next(base_tool_obj)
            for base_read in base_next:
                base_id = base_read.id
                base_seq = base_read.seq
                reads_to_compare[base_id] = [0] * number_of_tools
            n_tool = 0
            for tool in compared_tools_list:
                tool_next = next(tool)
                for tool_read in tool_next:
                    tool_id = tool_read.id
                    tool_seq = tool_read.seq
                    if tool_id == base_id and tool_seq != base_seq:
                        reads_to_compare[base_id][n_tool] = 1
                n_tool += 1
            yield reads_to_compare
        except:
            break

In [ ]:
# returns generator with read ID and trimmed sequence

def reads_diff_finder(main_fastq_file, fastq_file_to_compare):
    
    def read_fastq(fastq_open):
        head = fastq_open.readline().strip()
        seq = fastq_open.readline().strip()
        strain = fastq_open.readline().strip()
        qual = fastq_open.readline().strip()
        head_n = int(head[1:].split()[0].split(".")[1])
        return head, seq, strain, qual, head_n
    
    
    def reads_number(fastq_file):
        reads_number = 0
        with open(fastq_file) as fh:
            for line in fh:
                reads_number += 1
        return int(reads_number/4)
    
    
    main_file_len = reads_number(main_fastq_file)

    proscock = False

    with open(main_fastq_file) as fh1:    
        with open(fastq_file_to_compare) as fh2:
            for read in range(main_file_len):
                if not proscock:
                    head1, seq1, strain1, qual1, head1_n = read_fastq(fh1) 
                    head2, seq2, strain2, qual2, head2_n = read_fastq(fh2) 
                    if head2_n > head1_n: #if read was filtered it won't looking forwrd
                        proscock = True
                        continue
                    elif head2_n == head1_n and seq1 != seq2:
                        diff =  [head2, seq1.replace(seq2, "")]
                        yield diff
                        continue
                else:
                    head1, seq1, strain1, qual1, head1_n = read_fastq(fh1) 
                    if head2_n > head1_n: #if read was filtered it won't looking forwrd
                        proscock = True
                        continue
                    elif head2_n == head1_n:
                        if seq1 != seq2:
                            diff =  [head2, seq1.replace(seq2, "")]
                            yield diff
                            proscock = False
                            continue
                        else:
                            proscock = False
                            continue

In [ ]:
def reads_number(fastq_file):
    with open(fastq_file) as fh:
        reads_number = int(len(fh.readlines())/4)
    fh.close()
    return reads_number

In [ ]:
def write_diff_in_file(base_reads_file, compared_files, tools_list, file_to_write):
    for i in range(len(compared_files)):
        print(f"file {i}")
        
        if i == 0: 
            mode = "w"
        else: 
            mode = "a"
        
        raw_to_tool = reads_diff_finder(base_reads_file, compared_files[i])
        with open(file_to_write, mode) as fw:
            for p in raw_to_tool:
                n_reads += 1
                if n_reads % 100 == 0:
                    print(n_reads)
                header = p[0][1:].split()[0] + f"_{tools_list[i]}"
                sequence = p[1]
                fw.write(f">{header}\n{sequence}\n")
        fw.close()
    return f"Successfully written in {file_to_write}"

In [ ]:
### write all diffs in files
forward_diffs_all_tools = "/media/eternus1/nfs/projects/nkureev/mini_reads/all_tools_trimmed_1.fasta"
reverse_diffs_all_tools = "/media/eternus1/nfs/projects/nkureev/mini_reads/all_tools_trimmed_2.fasta"
tools_names = ["v2trim", "trimmomatic", "fastp", "trimgolore", "prinseq", "adapterremoval"]
tools_to_compare_1 = [reads_1_v2trim, read_1_trimmomatic, reads_1_fastp, reads_1_trimgalore, reads_1_prinseq, reads_1_adapterremoval]
tools_to_compare_2 = [reads_2_v2trim, read_2_trimmomatic, reads_2_fastp, reads_2_trimgalore, reads_2_prinseq, reads_2_adapterremoval]

# print(write_diff_in_file(raw_read_1, tools_to_compare_1, tools_names, forward_diffs_all_tools))
# print(write_diff_in_file(raw_read_2, tools_to_compare_2, tools_names, reverse_diffs_all_tools))

In [ ]:
base_reads_file = raw_read_2
compared_files = tools_to_compare_2
file_to_write = reverse_diffs_all_tools
tools_list = tools_names
n_reads = 0

for i in range(len(compared_files)):
    print(f"file {i}")

    if i == 0: 
        mode = "w"
    else: 
        mode = "a"

    raw_to_tool = reads_diff_finder(base_reads_file, compared_files[i])
    with open(file_to_write, mode) as fw:
        for read_header in raw_to_tool:
            n_reads += 1
            if n_reads % 10000 == 0:
                print(n_reads)
            header = read_header[0][1:].split()[0] + f"_{tools_list[i]}"
            sequence = p[1]
            fw.write(f">{header}\n{sequence}\n")
    fw.close()

In [ ]:
write_diff_in_file(raw_read_1, tools_to_compare_1, tools_names, forward_diffs_all_tools)

In [ ]:
raw_to_tool = reads_difffinder(base_reads_file, reads_difffinder)
forward = "1"

for i in raw_to_tool:
    header = i[0][1:].split()[0] + "_{forward}_{tool}"
    sequence = i[1]
    print(f">{header}\n{sequence}\n")

In [ ]:
main_fastq_file, fastq_file_to_compare = raw_read_1, read_1_trimmomatic

main_file_len = reads_number(main_fastq_file)

proscock = False

with open(main_fastq_file) as fh1:    
    with open(fastq_file_to_compare) as fh2:
        for read in range(main_file_len):
            if not proscock:
                head1 = fh1.readline().strip()
                seq1 = fh1.readline().strip()
                strain1 = fh1.readline()
                qual1 = fh1.readline()
                head1_n = int(head1[1:].split()[0].split(".")[1])
                #print(head1, seq1)

                head2 = fh2.readline().strip()
                seq2 = fh2.readline().strip()
                strain2 = fh2.readline()
                qual2 = fh2.readline()
                head2_n = int(head2[1:].split()[0].split(".")[1])
                #print(head2, seq2)
                
                if head2_n > head1_n: #if read was filtered it won't looking forwrd
                    proscock = True
                    continue
                elif head2_n == head1_n and seq1 != seq2:
                    diff =  [head2, seq1.replace(seq2, "")]
                    yield diff
                    continue
            else:
                head1 = fh1.readline().strip()
                seq1 = fh1.readline().strip()
                strain1 = fh1.readline()
                qual1 = fh1.readline()
                head1_n = int(head1[1:].split()[0].split(".")[1])
                if head2_n > head1_n: #if read was filtered it won't looking forwrd
                    proscock = True
                    continue
                elif head2_n == head1_n:
                    if seq1 != seq2:
                        diff =  [head2, seq1.replace(seq2, "")]
                        yield diff
                        proscock = False
                        continue
                    else:
                        proscock = False
                        continue

In [ ]:
raw_reads_gen_1, raw_reads_gen_2 = fastq_pe_reader(raw_read_1), fastq_pe_reader(raw_read_2)
v2trim_gen_1, v2trim_gen_2 = fastq_pe_reader(reads_1_v2trim), fastq_pe_reader(reads_2_v2trim)
trimmomatic_gen_1, trimmomatic_gen_2 = fastq_pe_reader(read_1_trimmomatic), fastq_pe_reader(read_2_trimmomatic)
fastp_gen_1, fastp_gen_2 = fastq_pe_reader(reads_1_fastp),  fastq_pe_reader(reads_2_fastp)
trimgalore_gen_1, trimgalore_gen_2 = fastq_pe_reader(reads_1_trimgalore), fastq_pe_reader(reads_2_trimgalore)
prinseq_gen_1, prinseq_gen_2 = fastq_pe_reader(reads_1_prinseq), fastq_pe_reader(reads_2_prinseq)
adapterremoval_gen_1, adapterremoval_gen_2 = fastq_pe_reader(reads_1_adapterremoval), fastq_pe_reader(reads_2_adapterremoval)

tools_to_compare_1 = [v2trim_gen_1, trimmomatic_gen_1, fastp_gen_1, trimgalore_gen_1, prinseq_gen_1, adapterremoval_gen_1]
tools_to_compare_2 = [v2trim_gen_2, trimmomatic_gen_2, fastp_gen_2, trimgalore_gen_2, prinseq_gen_2, adapterremoval_gen_2]
tools_names = ["v2trim", "trimmomatic", "fastp", "trimgolore", "prinseq", "adapterremoval"]
file_with_diffs_1 = "/media/eternus1/nfs/projects/nkureev/mini_reads/trimmed_parts_all_tools_1.fasta"
file_with_diffs_2 = "/media/eternus1/nfs/projects/nkureev/mini_reads/trimmed_parts_all_tools_2.fasta"

In [ ]:
compared_tools_list = []
for tool in tools_to_compare_1:
    tool_obj = (t for t in tool)
    compared_tools_list.append(tool_obj)
base_tool_obj = (t for t in raw_reads_gen_1)

number_of_tools = len(compared_tools_list)
while True:
    reads_to_compare = defaultdict(list)
    base_next = next(base_tool_obj)
    for base_read in base_next:
        base_id = base_read.id
        base_seq = base_read.seq
        print(base_id)
        reads_to_compare[base_id] = [0] * number_of_tools
    n_tool = 0
    for tool in compared_tools_list:
        tool_next = next(tool)
        for tool_read in tool_next:
            tool_id = tool_read.id
            tool_seq = tool_read.seq
            if tool_id == base_id and tool_seq != base_seq:
                reads_to_compare[base_id][n_tool] = 1
        n_tool += 1
    print(reads_to_compare)

In [ ]:
all_tools_compared_to_raw = reads_comparer(raw_reads_gen_1, tools_to_compare_1)

all_reads = 0
i = 0
for read in all_tools_compared_to_raw:
    for read_id, changes in read.items():
        all_reads += 1
        v2trim = changes[0]
        trimmomatic = changes[1]
        fastp = changes[2]
        if changes[0] == 1:
            i += 1
print(all_reads, i)

In [ ]:
# сделать фаста файлик
# хедер - id_tool
# сиквенс - (raw_read - tool_read)

# добавить next() чтбы вызывать строчки таблицы all_tools_compared_to_raw по очереди

raw_reads_gen = fastq_pe_reader(fastq_file1=raw_read_1, fastq_file2=raw_read_2)
v2trim_gen = fastq_pe_reader(fastq_file1=reads_1_v2trim, fastq_file2=reads_2_v2trim)
trimmomatic_gen = fastq_pe_reader(fastq_file1=read_1_trimmomatic, fastq_file2=read_2_trimmomatic)
fastp_gen = fastq_pe_reader(fastq_file1=reads_1_fastp, fastq_file2=reads_2_fastp)
trimgalore_gen = fastq_pe_reader(fastq_file1=reads_1_trimgalore, fastq_file2=reads_2_trimgalore)
prinseq_gen = fastq_pe_reader(fastq_file1=reads_1_prinseq, fastq_file2=reads_2_prinseq)
adapterremoval_gen = fastq_pe_reader(fastq_file1=reads_1_adapterremoval, fastq_file2=reads_2_adapterremoval)

raw_read_obj = (r for r in raw_reads_gen)
v2trim_read_obj = (r for r in v2trim_gen)
trimmomatic_read_obj = (r for r in trimmomatic_gen)
fastp_read_obj = (r for r in fastp_gen)
trimgalore_read_obj = (r for r in trimgalore_gen)
prinseq_read_obj = (r for r in prinseq_gen)
adapterremoval_read_obj = (r for r in adapterremoval_gen)
tools_to_compare = [v2trim_read_obj, trimmomatic_read_obj, fastp_read_obj, trimgalore_read_obj, prinseq_read_obj, adapterremoval_read_obj]

all_tools_compared_to_raw = reads_comparer(raw_read_obj, tools_to_compare)

fasta_trimmed_parts = "/media/eternus1/nfs/projects/nkureev/mini_reads/trimmed_parts.fasta"

with open(fasta_trimmed_parts, "w") as fw:
    for read in all_tools_compared_to_raw:
        for read_id, changes in read.items():
            v2trim = changes[0]
            trimmomatic = changes[1]
            fastp = changes[2]
            trimgalore = changes[3]
            prinseq = changes[4]
            adapterremoval = changes[5]
            tools = [v2trim, trimmomatic, fastp, trimgalore, prinseq, adapterremoval]
            tool_names = ["v2trim", "trimmomatic", "fastp", "trimgalore", "prinseq", "adapterremoval"]
            for i in range(len(tools)):
                if tools[i] == 1:
                    print(read_id)
                    for tool_read_id in tools_to_compare[i]:
                        for read in tool_read_id:
                            if read.id == read_id:
                                print(read.seq)


# подумать про сравнение тулов между собой

In [ ]:
raw_reads_gen = fastq_pe_reader(fastq_file1=raw_read_1, fastq_file2=raw_read_2)
v2trim_gen = fastq_pe_reader(fastq_file1=reads_1_v2trim, fastq_file2=reads_2_v2trim)
tools_list = [v2trim_gen]

one_compared_to_raw = reads_comparer(raw_reads_gen, tools_list)

i, j = 0, 0
for read in one_compared_to_raw:
    for read_id, changes in read.items():
        j += 1
        if changes[0] == 1:
            i += 1
print(j, i, i / j * 100)

In [ ]:
different_reads = []
losted_reads = []
i = 0
for header, seq in trimmomatic_reads.items():
    if header in v2trim_reads.keys():
        if seq != v2trim_reads[header]:
            different_reads.append(header)
    else:
        losted_reads.append(header)

In [ ]:
print(len(v2trim_reads[different_reads[0]]))
print(len(trimmomatic_reads[different_reads[0]]))

In [ ]:
i = 0
for header, seq in v2trim_reads.items():
    if len(seq) == 251:
        i += 1

In [ ]:
len(losted_reads)

In [ ]:
nontrimmed_by_trimmomatic = []
for key in different_reads:
    v2_seq = v2trim_reads[key]
    trim_seq = trimmomatic_reads[key]
    if len(v2_seq) > len(trim_seq):
        print(v2_seq.replace(trim_seq, ""))
    elif len(trim_seq) > len(v2_seq):
        nontrimmed_by_trimmomatic.append(trim_seq.replace(v2_seq, ""))

In [ ]:
j = 0
for i in nontrimmed_by_trimmomatic:
    print(i)

In [ ]:
fastq_file1 = raw_read_1
fastq_file2 = raw_read_2

with open(fastq_file1) as fh1:
        with open(fastq_file2) as fh2:
            while True:
                head1 = fh1.readline()
                if not head1: break
                seq1 = fh1.readline()
                strain1 = fh1.readline()
                qual1 = fh1.readline()
                fastq_obj1 = FastqObj(head1, seq1, strain1, qual1)

                head2 = fh2.readline()
                seq2 = fh2.readline()
                strain2 = fh2.readline()
                qual2 = fh2.readline()
                fastq_obj2 = FastqObj(head2, seq2, strain2, qual2, forvard=False)

In [ ]:
for i in fas

In [ ]:
## OLD STUFFF

In [ ]:
def create_last_table(reads_to_compare_table, reads_to_compare_dir, *compared_tools_reads_dirs):
    n_column = 1
    for tool in compared_tools_reads_dirs:
        reads_to_compare_table = between_tools_compare(reads_to_compare_table, reads_to_compare_dir, tool, n_column)
        n_column += 1
    return reads_to_compare_table

In [ ]:
# raw_reads compare with trimmomatic and v2_trim reads 0 - not equal, 1 - only one read equal, 2 - both reads equal
# raw_reads_compare[header] = [trimmomatic, v2trim]
def between_tools_compare(tool_to_compare_table, tool_to_compare_reads, compared_tool_reads, column_number):  
    
    for header, seq in tool_to_compare_reads.items():

        tc_forward = seq[0]
        tc_reverse = seq[1]

        if header in compared_tool_reads:
            ct_forward = compared_tool_reads[header][0]
            ct_reverse = compared_tool_reads[header][1]

            if tc_forward == ct_forward :
                if tc_reverse == ct_reverse:
                    tool_to_compare_table[header][column_number - 1] = 2
                else:
                    tool_to_compare_table[header][column_number - 1] = 1
            elif tc_reverse == ct_reverse:
                tool_to_compare_table[header][column_number - 1] = 1
    return tool_to_compare_table

In [ ]:
def reads_reader(reads_file_1, reads_file_2):    
    reads = {}
    header = None
    with open(reads_file_1) as fh1:
        while True:
            header = fh1.readline().split(" ")[0][1:]
            if not header:
                break
            seq = fh1.readline()
            strand = fh1.readline()
            strand_seq = fh1.readline()
            reads[header] = [seq]
            
    with open(reads_file_2) as fh2:
        while True:
            header = fh2.readline().split(" ")[0][1:]
            if not header:
                break
            seq = fh2.readline()
            strand = fh2.readline()
            strand_seq = fh2.readline()
            reads[header].append(seq)
    return reads

In [ ]:
def create_compare_table(reads_dict, n_columns):
    reads_to_compare = {}
    for key in raw_reads.keys():
        reads_to_compare[key] = [0]*n_columns
    return reads_to_compare

In [ ]:
raw_reads_to_compare_table = create_compare_table(raw_reads, 6)

In [ ]:
rr_table = create_last_table(raw_reads_to_compare_table, raw_reads, trimmomatic_reads, v2trim_reads)

In [ ]:
for i in range(10):
    print(i)

In [ ]:
def reads_reader(reads_file_1, reads_file_2 ):    
    reads = {}
    header = None
    with open(reads_file_1) as fh1:
        for i, line in enumerate(fh1):
            line = line.strip()
            if line.startswith("@SRR"):
                if header:
                    reads[header] = seq
                header = line.split()[0][1:]
                seq = []
                header_switcher = True
            elif header_switcher:
                seq.append(line)
                header_switcher = False
    if header:
        reads[header] = seq
        
    with open(reads_file_2) as fh2:
        for i, line in enumerate(fh2):
            line = line.strip()
            if line.startswith("@SRR"):
                if header:
                    reads[header].append(seq)
                header = line.split()[0][1:]
                seq = ""
                header_switcher = True
            elif header_switcher:
                seq += line
                header_switcher = False
    if header:
        reads[header].append(seq)
    return reads

In [ ]:
import glob

raw_read_1 = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/*_1.fastq") # 417842 ридов в сумме
raw_read_2 = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/*_2.fastq")

v2trim_1_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/v2trim/*.trim_1.fastq")
v2trim_2_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/v2trim/*.trim_2.fastq")

trimmomatic_1_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/trimmomatic/*pairedPE_1.fq")
trimmomatic_2_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/trimmomatic/*pairedPE_2.fq")

fastp_1_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/fastp/*_1.fq")
fastp_2_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/fastp/*_2.fq")

trimgalore_1_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/trim_galore/trim_galore*/*_1_val*.fq")
trimgalore_2_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/trim_galore/trim_galore*/*_2_val*.fq")

prinseq_1_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/prinseq/*good_out_R1.fastq")
prinseq_2_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/prinseq/*good_out_R2.fastq")

adapterremoval_1_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/AdapterRemoval/*_1.fq")
adapterremoval_2_reads = glob.glob("/media/eternus1/nfs/projects/nkureev/hiseq*/AdapterRemoval/*_2.fq")

In [ ]:
import os.path
diffinder = "/home/dzilov/scripts/reads_comparer/reads_diff_finder.py"
tools_list = ["v2trim", "trimmomatic", "fastp", "trimgolore", "prinseq", "adapterremoval"]
sequencers_list = ["hiseq_2000", "hiseq2500", "hiseq4000"]
tools_inputs_1 = raw_read_1
tools_inputs_2 = raw_read_2
tools_outputs_1 = [v2trim_1_reads, trimmomatic_1_reads, fastp_1_reads, trimgalore_1_reads, prinseq_1_reads, adapterremoval_1_reads]
tools_outputs_2 = [v2trim_2_reads, trimmomatic_2_reads, fastp_2_reads, trimgalore_2_reads, prinseq_2_reads, adapterremoval_2_reads]

for i in range(len(raw_read_1)):
    n = 0
    for tool in tools_outputs_1: 
        output_file = os.path.dirname(tool[i]) + f"/{tools_list[n]}_trimmed_parts_1.fasta"
        print(f"time {diffinder} -b {raw_read_1[i]} -o {output_file} -c {tool[i]} -t {tools_list[n]}" )
        n += 1

for i in range(len(raw_read_2)):
    n = 0
    for tool in tools_outputs_2: 
        output_file = os.path.dirname(tool[i]) + f"/{tools_list[n]}_trimmed_parts_2.fasta"
        print(f"time {diffinder} -b {raw_read_2[i]} -o {output_file} -c {tool[i]} -t {tools_list[n]}" )
        n += 1